In [1]:
import scanpy as sc, cell2location, numpy as np, pandas as pd
import sys, os, anndata

Global seed set to 0


In [ ]:
# Read Command Line Arguments
# Format: sample_path, inf_aver_path, fig_path
sample_id = sys.argv[-1]
sample_path, inf_aver_path, out_path = list(map(os.expanduser, sys.argv[1:-1]))

In [21]:
# Read in inf_aver csv and processed visium sample
inf_aver = pd.read_csv(inf_aver_path, index_col = 0)
# Note Ribosomal and MT genes have already been filtered
visium_obj = sc.read_h5ad(sample_path)

In [28]:
# Identify common features between inf_aver and visium_obj
common_feats = inf_aver.index.intersection(visium_obj.var_names)
# Subset to intersection
visium_obj = visium_obj[:, common_feats].copy()
inf_aver = inf_aver.loc[common_feats, :].copy()

In [31]:
# Initialise cell2location spatial mapping model
cell2location.models.Cell2location.setup_anndata(adata = visium_obj,
                                                batch_key = "orig.ident",
                                                labels_key = "opt_clust")
mapping_model = cell2location.models.Cell2location(visium_obj,
                                                  cell_state_df = inf_aver,
                                                  N_cells_per_location = 8,
                                                  detection_alpha = 20)
# Verify correct model setup
mapping_model.view_anndata_setup()

/home/dwzj28/miniconda3/envs/cell2loc_env/lib/python3.9/site-packages/scvi/data/_utils.py:114: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  warnings.warn(


Anndata setup with scvi-tools version 0.17.1.

Setup via `Cell2location.setup_anndata` with arguments:

{
│   'layer': None,
│   'batch_key': 'orig.ident',
│   'labels_key': 'opt_clust',
│   'categorical_covariate_keys': None,
│   'continuous_covariate_keys': None
}

         Summary Statistics         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃     Summary Stat Key     ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│          n_vars          │ 11938 │
│         n_cells          │ 1890  │
│         n_batch          │   1   │
│         n_labels         │   9   │
│ n_extra_categorical_covs │   0   │
│ n_extra_continuous_covs  │   0   │
└──────────────────────────┴───────┘

               Data Registry                
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Registry Key ┃    scvi-tools Location    ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      X       │          adata.X          │
│    batch     │ adata.obs['_scvi_batch']  │
│    labels    │ adata.obs['_scvi_labels'] │
│    ind_x     │   adata.obs['_indices']   │
└──────────────┴───────────────────────────┘

                     batch State Registry                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃     Source Location     ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['orig.ident'] │  ACH0010   │          0          │
└─────────────────────────┴────────────┴─────────────────────┘

                    labels State Registry                    
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃    Source Location     ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['opt_clust'] │     0      │          0          │
│                        │     1      │          1          │
│                        │     2      │          2          │
│                        │     3      │          3          │
│                        │     4      │          4          │
│                        │     5      │          5          │
│                        │     6      │          6          │
│                        │     7      │          7          │
│                        │     8      │          8          │
└────────────────────────┴────────────┴─────────────────────┘

In [55]:
import torch
torch.set_num_threads(8)
torch.get_num_threads()

8

In [56]:
# Run cell2location spatial mapping model
mapping_model.train(max_epochs = 500,
                   batch_size = None,
                   train_size = 1,
                   lr = 0.01)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dwzj28/miniconda3/envs/cell2loc_env/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:131: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
/home/dwzj28/miniconda3/envs/cell2loc_env/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1933: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 7/500:   1%|          | 6/500 [00:05<07:21,  1.12it/s, v_num=1, elbo_train=9.97e+6]

/home/dwzj28/miniconda3/envs/cell2loc_env/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [48]:
# Export Bayesian model posterior estimates on cell abundance to visium_obj
visium_obj = mapping_model.export_posterior(visium_obj,
                                           sample_kwargs = {"num_samples":1000})

# Save modified visium_obj results
visium_obj.write(out_path + sample_id + "_abundances.h5ad")

Sampling local variables, batch:   5%|▌         | 1/19 [00:20<06:06, 20.36s/it]


KeyboardInterrupt: 